In [36]:
import numpy as np 
import os 
import pandas as pd 
import re
import string
import seaborn as sns
from rake_nltk import Rake
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import vstack
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
import surprise
from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV

In [2]:
df_anime = pd.read_csv("../data/df_anime_export.csv")
df_profile = pd.read_csv("../data/df_profile_export.csv")
df_review = pd.read_csv("../data/df_review_export.csv")
df_an_pro_rev = pd.read_csv("../data/merged_rev_df_export.csv")
df_an_pro = pd.read_csv("../data/merged_df_export.csv")
df_main = pd.read_csv("../data/df_prof_rev_anime_clean.csv")

In [3]:
df_anime.head(1)

,Unnamed: 0,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,synopsis_processed
0,0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,"following participation inter-high , karasuno ..."


In [5]:
anime = df_anime[["uid", "title","genre","ranked","score","synopsis"]]

In [9]:
anime.rename(columns = {"ranked":"ranked_overall", "score":"score_overall"}, inplace = True)

F:\Anaconda\envs\learn-env\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [10]:
anime

,uid,title,genre,ranked_overall,score_overall,synopsis
0,28891,Haikyuu!! Second Season,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...",25.0,8.82,Following their participation at the Inter-Hig...
1,23273,Shigatsu wa Kimi no Uso,"['Drama', 'Music', 'Romance', 'School', 'Shoun...",24.0,8.83,Music accompanies the path of the human metron...
2,34599,Made in Abyss,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...",23.0,8.83,The Abyss—a gaping chasm stretching down into ...
3,5114,Fullmetal Alchemist: Brotherhood,"['Action', 'Military', 'Adventure', 'Comedy', ...",1.0,9.23,"""In order for something to be obtained, someth..."
4,31758,Kizumonogatari III: Reiketsu-hen,"['Action', 'Mystery', 'Supernatural', 'Vampire']",22.0,8.83,After helping revive the legendary vampire Kis...
...,...,...,...,...,...,...
16209,10075,Naruto x UT,"['Action', 'Comedy', 'Super Power', 'Martial A...",1728.0,7.50,All-new animation offered throughout UNIQLO cl...
16210,35828,Miira no Kaikata,"['Slice of Life', 'Comedy', 'Supernatural']",1727.0,7.50,High school student Sora Kashiwagi is accustom...
16211,10378,Shinryaku!? Ika Musume,"['Slice of Life', 'Comedy', 'Shounen']",1548.0,7.56,"After regaining her squid-like abilities, Ika ..."
16212,33082,Kingsglaive: Final Fantasy XV,['Action'],1544.0,7.56,"For years, the Niflheim Empire and the kingdom..."


In [19]:
anime_less = anime[["uid","title","genre"]]

In [16]:
review_score = df_main[["uid", "profile", "review_score","text_preprocessed"]]

In [17]:
review_score

,uid,profile,review_score,text_preprocessed
0,28891,skrn,7,"art : great , especially action critical momen..."
1,32935,skrn,8,"oh boy , finally stepped game good-but-not-gre..."
2,30276,skrn,7,one punch man reminds successful hollywood act...
3,4107,skrn,7,"movie recap first half tv series , % r use mat..."
4,4081,skrn,4,friend strongly recommended series insisted ne...
...,...,...,...,...
130514,33082,sensei69,5,* * * going spoiler free review ... * * * okay...
130515,33082,ssjtk,8,"n't listen critic , usually idea talking . mov..."
130516,33082,Ground_zero,9,final fantasy fan since old day although 's fa...
130517,33082,ClawViper,7,final fantasy fan following development final ...


## Begin combined method

In [21]:
# Collaborative Filtering - User-based CF
user_ratings = review_score.pivot(index='profile', columns='uid', values='review_score').fillna(0)

In [22]:
user_ratings

uid,1,5,6,7,8,15,16,17,18,19,...,40394,40420,40438,40477,40480,40489,40542,40693,40769,40807
profile,,,,,,,,,,,,,,,,,,,,,
-----noname-----,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
---SnowFlake---,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
---was-----,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--EYEPATCH--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--Mizu--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzyamuraihazz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzz-anime,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzzb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
# Compute user-user similarity using cosine similarity
user_similarity = cosine_similarity(user_ratings)

In [32]:
# Content-Based Filtering - Item-based CF
# Assume we have anime genres as content features
genres_vectorizer = TfidfVectorizer()
genres_matrix = genres_vectorizer.fit_transform(anime['genre'])

In [33]:
synopsis_vectorizer = TfidfVectorizer()
synopsis_matrix = synopsis_vectorizer.fit_transform(anime['synopsis'])

In [37]:
# Combine the content-based features (genres and synopsis)
combined_features_matrix = hstack([genres_matrix, synopsis_matrix])

In [38]:
# Compute item-item similarity using combined features
item_similarity = cosine_similarity(combined_features_matrix)

### Functions to use:

In [192]:
def get_collaborative_recommendations(user_id, num_recommendations):
    # Map user_id to integer position
    user_idx = user_ratings.index.get_loc(user_id)

    # Get top similar users to the given user
    similar_users = user_similarity[user_idx].argsort()[::-1][1:]

    # Find shows that similar users have reviewed highly but the given user hasn't reviewed
    user_rating = user_ratings.iloc[user_idx]
    unseen_shows = user_rating[user_rating == 0].index

    recommendations = []
    for user in similar_users:
        user_rating = user_ratings.iloc[user]
        shows_rated = user_rating[user_rating > 0].index
        shows_to_recommend = set(shows_rated) - set(unseen_shows)
        recommendations.extend(shows_to_recommend)
        if len(recommendations) >= num_recommendations:
            break
    
    # Get UIDs from show indices
    recommendations = [uid for uid in recommendations if uid in anime['uid'].values]
    return recommendations[:num_recommendations]

In [193]:
def get_content_based_recommendations(show_ids, num_recommendations):
    similar_shows = set()
    for show_id in show_ids:
        if show_id >= item_similarity.shape[0]:
            continue
        show_similarity = item_similarity[show_id]
        similar_shows.update(show_similarity.argsort()[::-1][:num_recommendations])
    
    # Get UIDs from show indices
    recommendations = [anime.loc[anime.index == show_idx]['uid'].values[0] for show_idx in similar_shows if show_idx in anime.index]
    return recommendations

In [194]:
def hybrid_recommendations(user_id, num_recommendations):
    collaborative_recs = get_collaborative_recommendations(user_id, num_recommendations)
    content_based_recs = get_content_based_recommendations(collaborative_recs, num_recommendations)

    # Combine recommendations
    combined_recs = list(set(collaborative_recs + content_based_recs))

    # Deduplicate and trim the final recommendations
    trimmed_recs = combined_recs[:num_recommendations]

    return trimmed_recs

In [198]:
# Example usage
user_id = 'LCSeixas'
num_recommendations = 5
recommendations = hybrid_recommendations(user_id, num_recommendations)

print("Hybrid Recommendations for User", user_id)
for show_id in recommendations:
    show_title = anime.loc[anime['uid'] == show_id]['title'].values
    if len(show_title) > 0:
        print(show_title[0])

Hybrid Recommendations for User LCSeixas
Chuunibyou demo Koi ga Shitai!: Kirameki no... Slapstick Noel


In [196]:
recommendations

[32935, 28891, 20583]

In [184]:
# Example usage
user_id = 'ssjtk'
num_recommendations = 5
recommendations = hybrid_recommendations(user_id, num_recommendations)

print("Hybrid Recommendations for User", user_id)
for show_id in recommendations:
    show_title = anime.loc[anime['uid'] == show_id]['title'].values
    if len(show_title) > 0:
        print(show_title[0])

Hybrid Recommendations for User ssjtk


In [185]:
recommendations

[]

In [177]:
# Example usage
user_id = 'skrn'
num_recommendations = 5
recommendations = hybrid_recommendations(user_id, num_recommendations)

print("Hybrid Recommendations for User", user_id)
for show_id in recommendations:
    show_title = anime.loc[anime['uid'] == show_id]['title'].values
    if len(show_title) > 0:
        print(show_title[0])

Hybrid Recommendations for User skrn
Cowboy Bebop
Puzzle & Dragons Cross
Cowboy Bebop: Tengoku no Tobira
Trigun
Nintama Rantarou no Uchuu Daibouken with Cosmic Front☆Next


In [178]:
recommendations

[1, 32772, 5, 6, 32773]